In [1]:
# LIBRARIES USED
import json
import torch
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import torch_pruning as tp
import torchvision.models as models
import caption
from torchsummary import summary

In [2]:
# MACROS
img = r"C:\Users\xiaomi\OneDrive\TUM\WS 2021-2022\Advanced Topics in Communication Electronics\image-captioning-on-pytorch-master\image-captioning-on-pytorch-master\img8.jpg"
word_map = r"C:\Users\xiaomi\OneDrive\TUM\WS 2021-2022\Advanced Topics in Communication Electronics\image-captioning-on-pytorch-master\image-captioning-on-pytorch-master\WORDMAP_coco_5_cap_per_img_5_min_word_freq.json"
model = r"C:\Users\xiaomi\OneDrive\TUM\WS 2021-2022\Advanced Topics in Communication Electronics\image-captioning-on-pytorch-master\image-captioning-on-pytorch-master\BEST_checkpoint_coco_5_cap_per_img_5_min_word_freq.pth.tar"
beam_size = 5
device = "cpu"

In [3]:
# INFERENCE FUNCTION
def inference(encoder, decoder, img, word_map = word_map, beam_size = beam_size):
    seq, alphas = caption.caption_image_beam_search(encoder, decoder, img, word_map, beam_size)
    alphas = torch.FloatTensor(alphas)
    words = [rev_word_map[ind] for ind in seq]
    sentence = ""
    for word in words:
        sentence = sentence + " " + word
    return sentence

In [4]:
# PRUNE FUNCTION
def prune_my_encoder(encoder):
    for name, module in encoder.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.ln_structured(module, name="weight", amount=0.05, n=2, dim=0)
            prune.remove(module, 'weight')
        elif isinstance(module, torch.nn.Linear):
            prune.ln_structured(module, name="weight", amount=0.05, n=2, dim=0)
            prune.remove(module, 'weight')
    eps = 1e-3
    with torch.no_grad():
        for name, module in myencoder.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                module.weight[abs(module.weight)<eps] = 0
            elif isinstance(module, torch.nn.Linear):
                module.weight[abs(module.weight)<eps] = 0            

In [5]:
# ENCODER CLASS
class Encoder(torch.nn.Module):
    """
    Encoder.
    """

    def __init__(self, encoded_image_size=14):
        super(Encoder, self).__init__()
        self.enc_image_size = encoded_image_size

        #resnet = torchvision.models.resnet101(pretrained=True)  # pretrained ImageNet ResNet-101
        resnet = models.resnet101()
        #resnet.load_state_dict(torch.load("resnet101-2.pth"))

        # Remove linear and pool layers (since we're not doing classification)
        modules = list(resnet.children())[:-2]
        self.resnet = torch.nn.Sequential(*modules)

        # Resize image to fixed size to allow input images of variable size
        self.adaptive_pool = torch.nn.AdaptiveAvgPool2d((encoded_image_size, encoded_image_size))

        self.fine_tune()

    def forward(self, images):
        """
        Forward propagation.

        :param images: images, a tensor of dimensions (batch_size, 3, image_size, image_size)
        :return: encoded images
        """
        out = self.resnet(images)  # (batch_size, 2048, image_size/32, image_size/32)
        out = self.adaptive_pool(out)  # (batch_size, 2048, encoded_image_size, encoded_image_size)
        out = out.permute(0, 2, 3, 1)  # (batch_size, encoded_image_size, encoded_image_size, 2048)
        return out

    def fine_tune(self, fine_tune=True):
        """
        Allow or prevent the computation of gradients for convolutional blocks 2 through 4 of the encoder.

        :param fine_tune: Allow?
        """
        for p in self.resnet.parameters():
            p.requires_grad = False
        # If fine-tuning, only fine-tune convolutional blocks 2 through 4
        for c in list(self.resnet.children())[5:]:
            for p in c.parameters():
                p.requires_grad = fine_tune

In [6]:
checkpoint = torch.load(model, map_location=device)
decoder = checkpoint['decoder']
decoder = decoder.to(device)
decoder.eval()
encoder = checkpoint['encoder']
encoder = encoder.to(device)
encoder.eval()
# Load word map (word2ix)
with open(word_map, 'r') as j:
    word_map = json.load(j)
rev_word_map = {v: k for k, v in word_map.items()}  # ix2word

C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:671: SourceChangeWarning: source code of class 'models.Encoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch

In [7]:
encoder.eval()

Encoder(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 2

In [8]:
myencoder = Encoder()
myencoder.eval()
myencoder = myencoder.to(device)
myencoder.eval()

Encoder(
  (resnet): Sequential(
    (0): QuantStub()
    (1): DeQuantStub()
    (2): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (6): Sequential(
      (0): Bottleneck(
        (quant): QuantStub()
        (dequant): DeQuantStub()
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [9]:
sd = encoder.state_dict()
sd2 = myencoder.state_dict()

In [10]:
for key in sd.keys():
    if key in sd2.keys():
        sd2[key] = sd[key]

In [11]:
myencoder.load_state_dict(sd2)

RuntimeError: Error(s) in loading state_dict for Encoder:
	size mismatch for resnet.6.0.conv1.weight: copying a param with shape torch.Size([256, 512, 1, 1]) from checkpoint, the shape in current model is torch.Size([64, 64, 1, 1]).
	size mismatch for resnet.6.0.bn1.weight: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for resnet.6.0.bn1.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for resnet.6.0.bn1.running_mean: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for resnet.6.0.bn1.running_var: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for resnet.6.0.conv2.weight: copying a param with shape torch.Size([256, 256, 3, 3]) from checkpoint, the shape in current model is tor

In [ ]:
inference(myencoder, decoder, img, word_map, beam_size)

In [ ]:
prune_my_encoder(myencoder)

In [ ]:
inference(myencoder, decoder, img, word_map, beam_size)

In [ ]:
total_encoder_sparsity = 0
den = 0
num = 0
for name, module in myencoder.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        print("Sparsity in {} (shape = {}): {:.2f}%".format
        (name, module.weight.shape,
        100. * float(torch.sum(module.weight == 0))/ float(module.weight.nelement())
        ))
        num += float(torch.sum(module.weight == 0))
        den += float(module.weight.nelement())
    elif isinstance(module, torch.nn.Linear):
        print("Sparsity in {} (shape = {}): {:.2f}%".format
        (name, module.weight.shape,
        100. * float(torch.sum(module.weight == 0))/ float(module.weight.nelement())
        ))
        num += float(torch.sum(module.weight == 0))
        den += float(module.weight.nelement())
total_encoder_sparsity = 100.*num/den        
print("Total sparsity in the pruned encoder: {:.2f}%".format(total_encoder_sparsity))        